<a href="https://colab.research.google.com/github/Ms-Gichuhi/Foot-ball-regression-analysis/blob/master/Week6_IP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Define The Question
You have been recruited as a football analyst in a company (Mchezopesa Ltd.) to accomplish the task below.


Provided two datasets containing football results and ranking data.

 A prediction result of a game between home team and away team, and on whether or not the game is friendly (include rank in your training).

Input: Home team, Away team, Tournament type (World cup, Friendly, Other)

Approach 1 : Polynomial Approach

What to train given:

Rank of home team

Rank of away team

Tournament type

Model 1: Predict how many goals the home team scores

Model 2: Predict how many goals the away team scores

Approach 2 : Logistic Approach

Figure out from the home team’s perspective if the game is a Win, Lose or Draw (W, L, D)

# Metrics of success
>polynomial regression model that predicts how many goals the home team scores.

>polynomial regression model that predicts how many goals the away team scores.

>logistic model that can predict who wins, loses or draws in a match with a high level of accuracy.

In [297]:
# Importing the required libraries
# ---
#
import pandas as pd
import numpy as np

In [298]:
#loading the dataset
fifa = pd.read_csv("fifa_ranking.csv")

results = pd.read_csv("results.csv")

In [299]:
#previewing our fifa datasets

fifa.head()

,rank,country_full,country_abrv,total_points,previous_points,rank_change,cur_year_avg,cur_year_avg_weighted,last_year_avg,last_year_avg_weighted,two_year_ago_avg,two_year_ago_weighted,three_year_ago_avg,three_year_ago_weighted,confederation,rank_date
0,1,Germany,GER,0.0,57,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,UEFA,1993-08-08
1,2,Italy,ITA,0.0,57,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,UEFA,1993-08-08
2,3,Switzerland,SUI,0.0,50,9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,UEFA,1993-08-08
3,4,Sweden,SWE,0.0,55,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,UEFA,1993-08-08
4,5,Argentina,ARG,0.0,51,5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,CONMEBOL,1993-08-08


In [300]:
#previewing the results dataset

results.head()

,date,home_team,away_team,home_score,away_score,tournament,city,country,neutral
0,1872-11-30,Scotland,England,0,0,Friendly,Glasgow,Scotland,False
1,1873-03-08,England,Scotland,4,2,Friendly,London,England,False
2,1874-03-07,Scotland,England,2,1,Friendly,Glasgow,Scotland,False
3,1875-03-06,England,Scotland,2,2,Friendly,London,England,False
4,1876-03-04,Scotland,England,3,0,Friendly,Glasgow,Scotland,False


In [301]:
#checking for missing values
print(fifa.isnull().sum())


rank                       0
country_full               0
country_abrv               0
total_points               0
previous_points            0
rank_change                0
cur_year_avg               0
cur_year_avg_weighted      0
last_year_avg              0
last_year_avg_weighted     0
two_year_ago_avg           0
two_year_ago_weighted      0
three_year_ago_avg         0
three_year_ago_weighted    0
confederation              0
rank_date                  0
dtype: int64


# Performing your EDA and  feature engineering 

In [302]:
print(fifa.duplicated().sum())

37


In [303]:
#Drop the duplicates 

fifa.drop_duplicates(keep='first', inplace = True)

In [304]:
fifa.shape

(57756, 16)

In [305]:
# properties of the fifa dataset
fifa.describe()

,rank,total_points,previous_points,rank_change,cur_year_avg,cur_year_avg_weighted,last_year_avg,last_year_avg_weighted,two_year_ago_avg,two_year_ago_weighted,three_year_ago_avg,three_year_ago_weighted
count,57756.000000,57756.000000,57756.000000,57756.000000,57756.000000,57756.000000,57756.000000,57756.000000,57756.000000,57756.000000,57756.000000,57756.000000
mean,101.618118,121.971151,332.339965,-0.009938,61.751362,61.751362,60.953507,30.476830,59.734768,17.920469,59.114466,11.822922
std,58.635096,260.479191,302.964176,5.802788,138.042006,138.042006,137.712945,68.856514,136.319119,40.895761,135.540135,27.108034
min,1.000000,0.000000,0.000000,-72.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,51.000000,0.000000,56.000000,-2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,101.000000,0.000000,272.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,152.000000,91.500000,525.000000,1.000000,30.447500,30.447500,26.400000,13.200000,21.250000,6.380000,20.795000,4.162500
max,209.000000,1775.030000,1920.000000,92.000000,1158.660000,1158.660000,1169.570000,584.790000,1159.710000,347.910000,1200.770000,240.150000


In [306]:
fifa.columns

Index(['rank', 'country_full', 'country_abrv', 'total_points',
       'previous_points', 'rank_change', 'cur_year_avg',
       'cur_year_avg_weighted', 'last_year_avg', 'last_year_avg_weighted',
       'two_year_ago_avg', 'two_year_ago_weighted', 'three_year_ago_avg',
       'three_year_ago_weighted', 'confederation', 'rank_date'],
      dtype='object')

In [307]:
#selecting columns we will use
fifa_df = fifa[['rank', 'country_full', 'rank_date']]
fifa_df.head()

,rank,country_full,rank_date
0,1,Germany,1993-08-08
1,2,Italy,1993-08-08
2,3,Switzerland,1993-08-08
3,4,Sweden,1993-08-08
4,5,Argentina,1993-08-08


In [308]:
fifa_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 57756 entries, 0 to 57792
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   rank          57756 non-null  int64 
 1   country_full  57756 non-null  object
 2   rank_date     57756 non-null  object
dtypes: int64(1), object(2)
memory usage: 1.8+ MB


In [309]:
fifa_df['rank_date'] = pd.to_datetime(fifa_df['rank_date'])
fifa_df['rank_date']

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


0       1993-08-08
1       1993-08-08
2       1993-08-08
3       1993-08-08
4       1993-08-08
           ...    
57788   2018-06-07
57789   2018-06-07
57790   2018-06-07
57791   2018-06-07
57792   2018-06-07
Name: rank_date, Length: 57756, dtype: datetime64[ns]

In [310]:
#renaming the country and date column to match the results datset for easier merging
fifa_df.columns = ['rank', 'country', 'date']
fifa_df.head()

,rank,country,date
0,1,Germany,1993-08-08
1,2,Italy,1993-08-08
2,3,Switzerland,1993-08-08
3,4,Sweden,1993-08-08
4,5,Argentina,1993-08-08


In [311]:
fifa_df.columns

Index(['rank', 'country', 'date'], dtype='object')

In [312]:
#cleaning up the results dataset fro merging
results.head()

,date,home_team,away_team,home_score,away_score,tournament,city,country,neutral
0,1872-11-30,Scotland,England,0,0,Friendly,Glasgow,Scotland,False
1,1873-03-08,England,Scotland,4,2,Friendly,London,England,False
2,1874-03-07,Scotland,England,2,1,Friendly,Glasgow,Scotland,False
3,1875-03-06,England,Scotland,2,2,Friendly,London,England,False
4,1876-03-04,Scotland,England,3,0,Friendly,Glasgow,Scotland,False


In [313]:
results.describe()

,home_score,away_score
count,40839.000000,40839.000000
mean,1.745709,1.188105
std,1.749145,1.405120
min,0.000000,0.000000
25%,1.000000,0.000000
50%,1.000000,1.000000
75%,2.000000,2.000000
max,31.000000,21.000000


In [314]:
#selecting columns we will use
results= results[['date', 'home_team', 'away_team', 'home_score', 'away_score', 'tournament', 'country']]
results.head()

,date,home_team,away_team,home_score,away_score,tournament,country
0,1872-11-30,Scotland,England,0,0,Friendly,Scotland
1,1873-03-08,England,Scotland,4,2,Friendly,England
2,1874-03-07,Scotland,England,2,1,Friendly,Scotland
3,1875-03-06,England,Scotland,2,2,Friendly,England
4,1876-03-04,Scotland,England,3,0,Friendly,Scotland


In [315]:
results.columns

Index(['date', 'home_team', 'away_team', 'home_score', 'away_score',
       'tournament', 'country'],
      dtype='object')

In [316]:
results.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40839 entries, 0 to 40838
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   date        40839 non-null  object
 1   home_team   40839 non-null  object
 2   away_team   40839 non-null  object
 3   home_score  40839 non-null  int64 
 4   away_score  40839 non-null  int64 
 5   tournament  40839 non-null  object
 6   country     40839 non-null  object
dtypes: int64(2), object(5)
memory usage: 2.2+ MB


In [317]:
#changing the date to datetime
results['date'] = pd.to_datetime(results['date'])
results['date']

0       1872-11-30
1       1873-03-08
2       1874-03-07
3       1875-03-06
4       1876-03-04
           ...    
40834   2019-07-18
40835   2019-07-18
40836   2019-07-19
40837   2019-07-19
40838   2019-07-20
Name: date, Length: 40839, dtype: datetime64[ns]

In [318]:
print(results['date'].describe())

print("................................................")
print(fifa_df['date'].describe())

count                   40839
unique                  15115
top       2012-02-29 00:00:00
freq                       66
first     1872-11-30 00:00:00
last      2019-07-20 00:00:00
Name: date, dtype: object
................................................
count                   57756
unique                    286
top       2017-04-06 00:00:00
freq                      211
first     1993-08-08 00:00:00
last      2018-06-07 00:00:00
Name: date, dtype: object


In [319]:
#dropping date before 1993 in the result dataframe since they do not have corresponding records in the fifa dataframe.
results_df = results[results["date"] > '1993-01-01']
results_df.shape


(23477, 7)

# **merging the datasets**

In [320]:
# Separate the date column into Year and Month of the fifa_ranking dataset
fifa_df['Year'] = fifa_df.date.dt.year

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [321]:
#calling out the year column in our fifa dataset for our final dataset
fifa_df.head()

,rank,country,date,Year
0,1,Germany,1993-08-08,1993
1,2,Italy,1993-08-08,1993
2,3,Switzerland,1993-08-08,1993
3,4,Sweden,1993-08-08,1993
4,5,Argentina,1993-08-08,1993


In [322]:
#calling out the year column in our results dataset for our final dataset
results_df['Year'] = results_df.date.dt.year


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [323]:
results_df.head()

,date,home_team,away_team,home_score,away_score,tournament,country,Year
17362,1993-01-02,Gabon,Burkina Faso,1,1,Friendly,Gabon,1993
17363,1993-01-02,Kuwait,Lebanon,2,0,Friendly,Kuwait,1993
17364,1993-01-03,Burkina Faso,Mali,1,0,Friendly,Gabon,1993
17365,1993-01-03,Gabon,Ghana,2,3,Friendly,Gabon,1993
17366,1993-01-08,Uganda,Tanzania,3,1,Friendly,Uganda,1993


In [324]:
# Merging the two datasets to obtain the hometeam  dataset 
home_df = pd.merge(fifa_df, results_df, how = 'inner', left_on = ['Year','country'], right_on = ['Year','home_team'])
home_df.head()

,rank,country_x,date_x,Year,date_y,home_team,away_team,home_score,away_score,tournament,country_y
0,1,Germany,1993-08-08,1993,1993-04-14,Germany,Ghana,6,1,Friendly,Germany
1,1,Germany,1993-08-08,1993,1993-10-13,Germany,Uruguay,5,0,Friendly,Germany
2,1,Germany,1993-08-08,1993,1993-11-17,Germany,Brazil,2,1,Friendly,Germany
3,5,Germany,1993-09-23,1993,1993-04-14,Germany,Ghana,6,1,Friendly,Germany
4,5,Germany,1993-09-23,1993,1993-10-13,Germany,Uruguay,5,0,Friendly,Germany


In [325]:
#from the merge above the country column matches the home team, we will drop the country_y  and rename country_x to country
#we will also rename the rank to hometeam rank
home_df['rank'] = home_df.rename({'rank': 'home_rank'}, axis = 1, inplace = True)

In [326]:
home_df['country_x'] = home_df.rename({'country_x': 'country'}, axis = 1, inplace = True)

In [327]:
home_df.drop(columns=['country_y'], inplace=True)

In [328]:
home_df.head()

,home_rank,country,date_x,Year,date_y,home_team,away_team,home_score,away_score,tournament,rank,country_x
0,1,Germany,1993-08-08,1993,1993-04-14,Germany,Ghana,6,1,Friendly,None,None
1,1,Germany,1993-08-08,1993,1993-10-13,Germany,Uruguay,5,0,Friendly,None,None
2,1,Germany,1993-08-08,1993,1993-11-17,Germany,Brazil,2,1,Friendly,None,None
3,5,Germany,1993-09-23,1993,1993-04-14,Germany,Ghana,6,1,Friendly,None,None
4,5,Germany,1993-09-23,1993,1993-10-13,Germany,Uruguay,5,0,Friendly,None,None


In [329]:
#picking out the columns we need from thr hometeam df
home_df= home_df[['home_rank', 'Year', 'home_team', 'away_team', 'home_score', 'away_score']]
home_df.head()

,home_rank,Year,home_team,away_team,home_score,away_score
0,1,1993,Germany,Ghana,6,1
1,1,1993,Germany,Uruguay,5,0
2,1,1993,Germany,Brazil,2,1
3,5,1993,Germany,Ghana,6,1
4,5,1993,Germany,Uruguay,5,0


In [330]:
# Merging the two datasets to obtain the awayteam  dataset 
away_df = pd.merge(fifa_df, results_df, how = 'inner', left_on = ['Year','country'], right_on = ['Year','away_team'])
away_df.head()

,rank,country_x,date_x,Year,date_y,home_team,away_team,home_score,away_score,tournament,country_y
0,1,Germany,1993-08-08,1993,1993-03-24,Scotland,Germany,0,1,Friendly,Scotland
1,1,Germany,1993-08-08,1993,1993-06-10,Brazil,Germany,3,3,USA Cup,United States
2,1,Germany,1993-08-08,1993,1993-06-13,United States,Germany,3,4,USA Cup,United States
3,1,Germany,1993-08-08,1993,1993-06-19,England,Germany,1,2,USA Cup,United States
4,1,Germany,1993-08-08,1993,1993-09-22,Tunisia,Germany,1,1,Friendly,Tunisia


In [331]:
#from the merge above the country column matches the away team, we will drop the country_y  and rename country_x to country
#we will also rename the rank to awayteam rank
away_df['rank'] = away_df.rename({'rank': 'away_rank'}, axis = 1, inplace = True)

In [332]:
away_df['country_x'] = away_df.rename({'country_x': 'country'}, axis = 1, inplace = True)

In [333]:
away_df.drop(columns=['country_y'], inplace=True)

In [334]:
away_df.head()

,away_rank,country,date_x,Year,date_y,home_team,away_team,home_score,away_score,tournament,rank,country_x
0,1,Germany,1993-08-08,1993,1993-03-24,Scotland,Germany,0,1,Friendly,None,None
1,1,Germany,1993-08-08,1993,1993-06-10,Brazil,Germany,3,3,USA Cup,None,None
2,1,Germany,1993-08-08,1993,1993-06-13,United States,Germany,3,4,USA Cup,None,None
3,1,Germany,1993-08-08,1993,1993-06-19,England,Germany,1,2,USA Cup,None,None
4,1,Germany,1993-08-08,1993,1993-09-22,Tunisia,Germany,1,1,Friendly,None,None


In [335]:
#picking out the columns we need from thr awayteam df
away_df= away_df[['away_rank', 'Year', 'away_team', 'home_team', 'away_score', 'tournament', 'home_score']]
away_df.head()

,away_rank,Year,away_team,home_team,away_score,tournament,home_score
0,1,1993,Germany,Scotland,1,Friendly,0
1,1,1993,Germany,Brazil,3,USA Cup,3
2,1,1993,Germany,United States,4,USA Cup,3
3,1,1993,Germany,England,2,USA Cup,1
4,1,1993,Germany,Tunisia,1,Friendly,1


In [336]:
#now merging the hometeam and awayteams dataset
df = pd.merge(home_df, away_df)
df.head()

,home_rank,Year,home_team,away_team,home_score,away_score,away_rank,tournament
0,1,1993,Germany,Ghana,6,1,48,Friendly
1,1,1993,Germany,Ghana,6,1,42,Friendly
2,1,1993,Germany,Ghana,6,1,40,Friendly
3,1,1993,Germany,Ghana,6,1,41,Friendly
4,1,1993,Germany,Ghana,6,1,37,Friendly


In [337]:
df['status'] = df['home_score'] - df['away_score']
def status(x):
    if x > 0:
        return 'Win'
    elif x < 0:
        return 'Loss'
    else:
        return 'Draw'
      
df['status'] = df.status.apply(lambda y: status(y))
df.tail()

#we created a new column for trhe status of the hometeam ,was it a win ,loss or draw?
#it is the difference btetween home scores and away scores

,home_rank,Year,home_team,away_team,home_score,away_score,away_rank,tournament,status
2356803,195,2018,Gibraltar,Armenia,2,6,91,UEFA Nations League,Loss
2356804,195,2018,Gibraltar,Armenia,2,6,91,UEFA Nations League,Loss
2356805,195,2018,Gibraltar,Armenia,2,6,98,UEFA Nations League,Loss
2356806,195,2018,Gibraltar,Armenia,2,6,98,UEFA Nations League,Loss
2356807,195,2018,Gibraltar,Armenia,2,6,100,UEFA Nations League,Loss


In [338]:
#df['tournament'].value_counts()
# we will specify the Tournament type (World cup, Friendly, Other)
def tournament(class_type):
    if class_type == 'Friendly':
        return 'Friendly'
    elif class_type == 'FIFA World Cup':
        return 'World cup'
    else:
        return 'Other'
df['tournament'] = df.tournament.apply(lambda x: tournament(x))

In [339]:
df['tournament'].value_counts()

Other        1436701
Friendly      881909
World cup      38198
Name: tournament, dtype: int64

In [340]:
df.to_csv('fifa_result.csv')

# **Check of multicollinearity**

In [341]:
df.columns

Index(['home_rank', 'Year', 'home_team', 'away_team', 'home_score',
       'away_score', 'away_rank', 'tournament', 'status'],
      dtype='object')

In [342]:

#to focus on our independent variables only.
independent_only = df[['home_rank', 'home_score', 'away_score', 'away_rank']] 

#displaying the correlations between the variables
correlations = independent_only.corr()

correlations

,home_rank,home_score,away_score,away_rank
home_rank,1.000000,-0.140598,0.223114,0.469410
home_score,-0.140598,1.000000,-0.161781,0.319172
away_score,0.223114,-0.161781,1.000000,-0.179735
away_rank,0.469410,0.319172,-0.179735,1.000000


In [343]:
#calculating the VIF from the correlation above
pd.DataFrame(np.linalg.inv(correlations.values), index = correlations.index, columns=correlations.columns)

# If VIF = 1, a variable  is not correlated with any other independent variable. 
#As a rule of thumb, multicollinearity is a potential problem when VIF is greater than 4; and,
# a serious problem when it is greater than 10. The output above shows a VIF of:
# home-rank = 1.64,away_rank = 1.75,home_score = 1.26, away_score = 1.18
# which indicates some multicollinearity,not too much so we do nothing about it.

,home_rank,home_score,away_score,away_rank
home_rank,1.646532,0.477592,-0.471649,-1.010104
home_score,0.477592,1.266101,-0.015141,-0.631012
away_score,-0.471649,-0.015141,1.181615,0.438606
away_rank,-1.010104,-0.631012,0.438606,1.754387


# **Building the model**

# **polynomial approach**


> What to train given:

Rank of home team

Rank of away team

Tournament type

Model 1: Predict how many goals the home team scores.



In [344]:
#predicting the home scores
import matplotlib.pyplot as plt

from sklearn.preprocessing import PolynomialFeatures

from sklearn.model_selection import train_test_split

# Import LinearRegression method from sklearn linear_model library
from sklearn.linear_model import LinearRegression

In [345]:
x = (df[['home_rank', 'away_rank', 'tournament']])
y = df['home_score']

print(x.shape)
print(y.shape)

(2356808, 3)
(2356808,)


In [346]:
#Encoding categorical variables
x = pd.get_dummies(x, drop_first=True)

In [347]:
x = np.array(x)

In [348]:
#fitting the model
regressor = LinearRegression()
regressor.fit(x, y)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [349]:
#Establish training and testing sets

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 0)

In [350]:
# Fit polynomial Regression to the dataset
poly_reg = PolynomialFeatures(degree = 4) 
x_poly = poly_reg.fit_transform(x)

In [351]:
# Fit linear regression to our polynomial regression model
reg = LinearRegression()
reg.fit(x_poly, y)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [352]:
#our dataframe seems so huge to work with , we therefore picked a base model [20, 4, 2, 1]Home rank: 20,Away rank: 4,Friendly: 1 world cup:2
# predict using linear regression
lin_pred = regressor.predict([[20, 4, 2, 1]])

print("Linear Prediction: %d" %lin_pred)

print ('.....................................')

# predict using polynomial regression
poly_pred = reg.predict(poly_reg.fit_transform([[20, 4, 2, 1]]))

print("Polynomial Prediction: %d" %poly_pred)


Linear Prediction: 1
.....................................
Polynomial Prediction: 1


Model 2: Predict how many goals the away team scores.

In [353]:
#predicting the away team scores
X= (df[['away_rank', 'home_rank', 'tournament']])
y = df['away_score']

print(X.shape)
print(y.shape)

(2356808, 3)
(2356808,)


In [354]:
#Encoding categorical variables
X = pd.get_dummies(X, drop_first=True)

In [355]:
X = np.array(X)

In [356]:
#fitting the model
regressor = LinearRegression()
regressor.fit(X, y)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [357]:
#Establish training and testing sets

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

In [358]:
# Fit polynomial Regression to the dataset
poly_reg = PolynomialFeatures(degree = 4) 
X_poly = poly_reg.fit_transform(X)

In [359]:
# Fit linear regression to our polynomial regression model
reg = LinearRegression()
reg.fit(X_poly, y)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [360]:
#our dataframe seems so huge to work with , we therefore picked a base model [20, 4, 2, 1]Home rank: 20,Away rank: 4,Friendly: 1 world cup:2
# predict using linear regression
lin_pred = regressor.predict([[20, 4, 2, 1]])

print("Linear Prediction: %d" %lin_pred)

print ('.....................................')

# predict using polynomial regression
poly_pred = reg.predict(poly_reg.fit_transform([[20, 4, 2, 1]]))

print("Polynomial Prediction: %d" %poly_pred)

Linear Prediction: 1
.....................................
Polynomial Prediction: 0


# logistic approach



> Figure out from the home team’s perspective if the game is a Win, Lose or Draw (W, L, D)



In [361]:
df.head()

,home_rank,Year,home_team,away_team,home_score,away_score,away_rank,tournament,status
0,1,1993,Germany,Ghana,6,1,48,Friendly,Win
1,1,1993,Germany,Ghana,6,1,42,Friendly,Win
2,1,1993,Germany,Ghana,6,1,40,Friendly,Win
3,1,1993,Germany,Ghana,6,1,41,Friendly,Win
4,1,1993,Germany,Ghana,6,1,37,Friendly,Win


In [362]:
#predicting the home team status: win,loss or draw
X= (df[['home_score', 'home_rank', 'tournament']])
Y = df['status']

print(X.shape)
print(Y.shape)

(2356808, 3)
(2356808,)


In [363]:
#Encoding categorical variables
X = pd.get_dummies(X, drop_first=True)

In [364]:
X = np.array(X)

In [365]:
#splitting into training and test sets
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=42)

In [366]:
# Fitting our model

from sklearn.linear_model import LogisticRegression

LogReg = LogisticRegression()
LogReg.fit(X_train, Y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [367]:
# Using our model to make a prediction

y_pred = LogReg.predict(X_test)

In [368]:
# Measuring the accuracy of the model

from sklearn.metrics import accuracy_score
print("accuracy of the model is", {accuracy_score(Y_test, y_pred)})

accuracy of the model is {0.6543435123464909}
